# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [3]:
# import libraries
import pandas as pd
import numpy as np
#database
from sqlalchemy import create_engine
#nltk
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import word_tokenize,sent_tokenize
nltk.download('punkt')
##sklearn
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_multilabel_classification
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
  

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [4]:
# load data from database
cnx = create_engine('sqlite:///etlpipeline2.db').connect()
df = pd.read_sql_table('etlpipeline2', cnx)

In [5]:
df = df.reset_index()

In [6]:
len(df)

26216

In [7]:
df.head()

,index,id,message,original,genre,related,request,offer,aid_related,medical_help,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,...,0,0,1,0,1,0,0,0,0,0
2,2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
X =  df.message.values
X

array(['Weather update - a cold front from Cuba that could pass over Haiti',
       'Is the Hurricane over or is it not over',
       'Looking for someone but no name', ...,
       "Proshika, operating in Cox's Bazar municipality and 5 other unions, Ramu and Chokoria, assessment, 5 kg rice, 1,5 kg lentils to 700 families.",
       'Some 2,000 women protesting against the conduct of the elections were teargassed as they tried to converge on the local electoral commission offices in the southern oil city of Port Harcourt.',
       'A radical shift in thinking came about as a result of this meeting, recognizing that HIV/AIDS is at the core of the humanitarian crisis and identifying the crisis itself as a function of the HIV/AIDS pandemic.'], dtype=object)

In [9]:
Y=  df.iloc[:,4:]
Y=(Y.astype('str'))
Y.head()

,genre,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,direct,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,direct,1,0,0,1,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,direct,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,direct,1,1,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,direct,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
Y.tail(n=10)

,genre,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
26206,news,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26207,news,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
26208,news,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26209,news,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26210,news,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26211,news,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26212,news,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26213,news,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26214,news,1,0,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
26215,news,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
Y.columns

Index(['genre', 'related', 'request', 'offer', 'aid_related', 'medical_help',
       'medical_products', 'search_and_rescue', 'security', 'military',
       'child_alone', 'water', 'food', 'shelter', 'clothing', 'money',
       'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
       'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
       'other_weather', 'direct_report'],
      dtype='object')

### 2. Write a tokenization function to process your text data

In [12]:
#test tokenize with one word
review = re.sub('[^a-zA-Z]', ' ', X[3])
review = review.lower()
review = review.split()
ps = PorterStemmer()
all_stopwords = stopwords.words('english')
all_stopwords.remove('not')
review = [ps.stem(word) for word in review if not word in set(all_stopwords)]
review = ' '.join(review)
print(review)
review = word_tokenize(review)
print(review)

un report leogan destroy hospit st croix function need suppli desper
['un', 'report', 'leogan', 'destroy', 'hospit', 'st', 'croix', 'function', 'need', 'suppli', 'desper']


In [19]:
def tokenize(text):
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")
    review = re.sub('[^a-zA-Z0-9]', ' ', text)
    review = review.lower()
    review = review.split()
    ps = PorterStemmer()
    all_stopwords = stopwords.words('english')
    all_stopwords.remove('not')
    review = [ps.stem(word) for word in review if not word in set(all_stopwords)]
    review = ' '.join(review)
    review = word_tokenize(review)

    return review

In [20]:
tokenize(X[3])

['un',
 'report',
 'leogan',
 '80',
 '90',
 'destroy',
 'hospit',
 'st',
 'croix',
 'function',
 'need',
 'suppli',
 'desper']

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [13]:
pipeline = Pipeline([('vect',CountVectorizer(tokenizer=tokenize)),
                    ('tfidf',TfidfTransformer()),
                    ('clf',MultiOutputClassifier(RandomForestClassifier())
                    )])

In [14]:
np.unique(Y)

array(['0', '1', '2', 'direct', 'news', 'social'], dtype=object)

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X,Y)

In [16]:
pipeline.fit(X_train,y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [17]:
y_pred = pipeline.predict(X_test)

In [18]:
for index, col in enumerate(Y.columns):
    print(col)
    print(classification_report(y_test[col], y_pred[:,index]))

avg = (y_pred == y_test).mean().mean()
print("Accuracy Overall:\n", avg)

genre
             precision    recall  f1-score   support

     direct       0.88      0.95      0.91      2715
       news       0.95      0.92      0.93      3265
     social       0.97      0.77      0.86       574

avg / total       0.92      0.92      0.92      6554

related
             precision    recall  f1-score   support

          0       0.61      0.47      0.53      1533
          1       0.85      0.91      0.87      4970
          2       0.33      0.39      0.36        51

avg / total       0.79      0.80      0.79      6554

request
             precision    recall  f1-score   support

          0       0.89      0.98      0.93      5422
          1       0.80      0.43      0.56      1132

avg / total       0.88      0.88      0.87      6554

offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6524
          1       0.00      0.00      0.00        30

avg / total       0.99      1.00      0.99      6554

aid_re

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


             precision    recall  f1-score   support

          0       0.76      0.86      0.81      3825
          1       0.76      0.62      0.68      2729

avg / total       0.76      0.76      0.75      6554

medical_help
             precision    recall  f1-score   support

          0       0.93      0.99      0.96      6022
          1       0.60      0.12      0.19       532

avg / total       0.90      0.92      0.90      6554

medical_products
             precision    recall  f1-score   support

          0       0.96      1.00      0.98      6232
          1       0.75      0.10      0.18       322

avg / total       0.95      0.95      0.94      6554

search_and_rescue
             precision    recall  f1-score   support

          0       0.98      1.00      0.99      6393
          1       0.50      0.06      0.10       161

avg / total       0.97      0.98      0.97      6554

security
             precision    recall  f1-score   support

          0       0.98      1

### 6. Improve your model
Use grid search to find better parameters. 

In [20]:
pipeline.get_params

<bound method Pipeline.get_params of Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])>

In [23]:
from sklearn.model_selection import GridSearchCV

parameters = {
        'vect__max_df':[0.5,0.9],
        'clf__estimator__n_estimators': [20, 40]
    }


cv = GridSearchCV(pipeline, param_grid=parameters, verbose=10, n_jobs=4)
cv.fit(X_train, y_train)

Fitting 3 folds for each of 4 candidates, totalling 12 fits
[CV] clf__estimator__n_estimators=20, vect__max_df=0.5 ...............
[CV] clf__estimator__n_estimators=20, vect__max_df=0.5 ...............
[CV] clf__estimator__n_estimators=20, vect__max_df=0.5 ...............
[CV] clf__estimator__n_estimators=20, vect__max_df=0.9 ...............
[CV]  clf__estimator__n_estimators=20, vect__max_df=0.5, score=0.23863289594140982, total= 6.8min
[CV] clf__estimator__n_estimators=20, vect__max_df=0.9 ...............
[CV]  clf__estimator__n_estimators=20, vect__max_df=0.5, score=0.22978333841928594, total= 6.8min
[CV] clf__estimator__n_estimators=20, vect__max_df=0.9 ...............
[CV]  clf__estimator__n_estimators=20, vect__max_df=0.5, score=0.23619163869392737, total= 6.8min
[CV] clf__estimator__n_estimators=40, vect__max_df=0.5 ...............
[CV]  clf__estimator__n_estimators=20, vect__max_df=0.9, score=0.23619163869392737, total= 6.9min
[CV] clf__estimator__n_estimators=40, vect__max_df=

[Parallel(n_jobs=4)]: Done   5 tasks      | elapsed: 16.7min


[CV] clf__estimator__n_estimators=40, vect__max_df=0.9 ...............
[CV]  clf__estimator__n_estimators=40, vect__max_df=0.5, score=0.24580408910588952, total=12.2min


[Parallel(n_jobs=4)]: Done   7 out of  12 | elapsed: 22.5min remaining: 16.1min


[CV] clf__estimator__n_estimators=40, vect__max_df=0.9 ...............
[CV]  clf__estimator__n_estimators=40, vect__max_df=0.5, score=0.2380225816295392, total=12.1min
[CV] clf__estimator__n_estimators=40, vect__max_df=0.9 ...............
[CV]  clf__estimator__n_estimators=40, vect__max_df=0.5, score=0.24076899603295698, total=12.2min


[Parallel(n_jobs=4)]: Done   9 out of  12 | elapsed: 30.8min remaining: 10.3min


[CV]  clf__estimator__n_estimators=40, vect__max_df=0.9, score=0.2465669819957278, total=12.3min
[CV]  clf__estimator__n_estimators=40, vect__max_df=0.9, score=0.2422947818126335, total=10.1min
[CV]  clf__estimator__n_estimators=40, vect__max_df=0.9, score=0.23817516020750687, total=10.1min


[Parallel(n_jobs=4)]: Done  12 out of  12 | elapsed: 33.6min finished


GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=4,
       param_grid={'vect__max_df': [0.5, 0.9], 'clf__estimator__n_estimators': [20, 40]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=10)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [24]:
y_pred = pipeline.predict(X_test)
for index, col in enumerate(Y.columns):
    print(col)
    print(classification_report(y_test[col], y_pred[:,index]))

avg = (y_pred == y_test).mean().mean()
print("Accuracy Overall:\n", avg)

genre
             precision    recall  f1-score   support

     direct       0.88      0.95      0.91      2715
       news       0.95      0.92      0.93      3265
     social       0.97      0.77      0.86       574

avg / total       0.92      0.92      0.92      6554

related
             precision    recall  f1-score   support

          0       0.61      0.47      0.53      1533
          1       0.85      0.91      0.87      4970
          2       0.33      0.39      0.36        51

avg / total       0.79      0.80      0.79      6554

request
             precision    recall  f1-score   support

          0       0.89      0.98      0.93      5422
          1       0.80      0.43      0.56      1132

avg / total       0.88      0.88      0.87      6554

offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6524
          1       0.00      0.00      0.00        30

avg / total       0.99      1.00      0.99      6554

aid_re

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


             precision    recall  f1-score   support

          0       0.76      0.86      0.81      3825
          1       0.76      0.62      0.68      2729

avg / total       0.76      0.76      0.75      6554

medical_help
             precision    recall  f1-score   support

          0       0.93      0.99      0.96      6022
          1       0.60      0.12      0.19       532

avg / total       0.90      0.92      0.90      6554

medical_products
             precision    recall  f1-score   support

          0       0.96      1.00      0.98      6232
          1       0.75      0.10      0.18       322

avg / total       0.95      0.95      0.94      6554

search_and_rescue
             precision    recall  f1-score   support

          0       0.98      1.00      0.99      6393
          1       0.50      0.06      0.10       161

avg / total       0.97      0.98      0.97      6554

security
             precision    recall  f1-score   support

          0       0.98      1

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [26]:
import pickle
pickle.dump(cv, open("classifier.pkl", 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.